In [1]:
import pandas as pd
import numpy as np

In [2]:
a = [np.array([0, 0]), np.array([1, -1]),np.array([1, 1]), np.array([-1, 1])]
a

[array([0, 0]), array([ 1, -1]), array([1, 1]), array([-1,  1])]

In [3]:
s = pd.Series(a)
s

0     [0, 0]
1    [1, -1]
2     [1, 1]
3    [-1, 1]
dtype: object

In [8]:
s[1][1]

-1

In [10]:
df = pd.DataFrame(s)
df

,0
0,"[0, 0]"
1,"[1, -1]"
2,"[1, 1]"
3,"[-1, 1]"


In [34]:
np.array(df[0].tolist())[:, 0]

array([ 0,  1,  1, -1])